In [92]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.special import softmax, log_softmax

In [93]:
data = pd.read_csv('../Data/choices_exp1_ext.csv')

In [166]:
def loglik(params, data, y, self_x, other_x, self_y, other_y,
          indicators_x, indicators_y):

    sigma = params[-1]
    
    if len(params)>1:
        regs = params[0:-1]
        
        lli = np.dot(indicators_x,np.array(regs))
        rli = np.dot(indicators_y,np.array(regs))
        
        uleft = sigma*((1-lli)*self_x+lli*other_x)
        uright = sigma*((1-rli)*self_y+rli*other_y)
        
    else:
        uleft = sigma*self_x
        uright = sigma*self_y
    
    probs = log_softmax([uleft,uright], axis=0).T[:,0]*y + \
            log_softmax([uleft,uright], axis=0).T[:,1]*(1-y)


    
    return -1*np.sum(probs) 

In [96]:
y = data['choice_x']
self_x = data['self_x']
self_y = data['self_y']
other_x = data['other_x']
other_y = data['other_y']
data['g']=1
indicators_x = data[['g']]
indicators_y = data[['g']]

In [134]:
params0 = [np.random.uniform(-1,1),np.random.uniform(0,0.01)]
res = minimize(loglik, params0, args=(data),
              method='BFGS', options={'disp': False, 'gtol':1e-7*len(data)})

In [135]:
res

      fun: 7289.0477677649105
 hess_inv: array([[ 1.94539824e-05, -9.95260372e-08],
       [-9.95260372e-08,  2.60774070e-08]])
      jac: array([-0.00018311, -0.00085449])
  message: 'Optimization terminated successfully.'
     nfev: 48
      nit: 10
     njev: 16
   status: 0
  success: True
        x: array([0.16595335, 0.01278581])

In [154]:
indicators_x = data[['s_x','r_x','q','v']]
indicators_y = data[['s_y','r_y','q','v']]

In [155]:
params0 = [np.random.uniform(-1,1),np.random.uniform(-1,1),
           np.random.uniform(-1,1),np.random.uniform(-1,1),
           np.random.uniform(0,0.01)]
res = minimize(loglik, params0, args=(data),
              method='BFGS', options={'disp': False, 'gtol':1e-7*len(data)})

In [156]:
res

      fun: 7068.38455778586
 hess_inv: array([[ 8.44162486e-05,  2.79690327e-05, -5.70166812e-05,
        -5.55911278e-05, -5.36880303e-08],
       [ 2.79690327e-05,  8.25707450e-05, -5.41881088e-05,
        -5.60923524e-05, -1.51796263e-07],
       [-5.70166812e-05, -5.41881088e-05,  1.10435336e-04,
         5.76735780e-05, -2.52270444e-08],
       [-5.55911278e-05, -5.60923524e-05,  5.76735780e-05,
         1.13889332e-04,  3.51828179e-08],
       [-5.36880303e-08, -1.51796263e-07, -2.52270444e-08,
         3.51828179e-08,  2.75343508e-08]])
      jac: array([-6.10351562e-05,  1.22070312e-04,  6.10351562e-05,  6.10351562e-05,
       -1.83105469e-03])
  message: 'Optimization terminated successfully.'
     nfev: 132
      nit: 15
     njev: 22
   status: 0
  success: True
        x: array([ 0.06024037,  0.24828066,  0.07248044, -0.04791563,  0.01314331])

In [ ]:
lli

In [ ]:
0.5*(1-lli)*self_x

In [ ]:
self_x

In [143]:
res.x

array([ 0.06024039,  0.24828067,  0.07248043, -0.04791564,  0.01314331])

In [145]:
loglik(res.x,data)/len(data)

0.34720427143068305

In [147]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, train_size=0.7, stratify=data['sid'], random_state=181, shuffle=True)

In [149]:
loglik(res.x,test)

7068.384557785846

In [178]:
def estim(data, label_y, label_self_x, label_other_x, label_self_y,
          label_other_y, label_indicators_x, label_indicators_y, params=None):
    
    y = data[label_y]
    self_x = data[label_self_x]
    self_y = data[label_self_y]
    other_x = data[label_other_x]
    other_y = data[label_other_y]
    indicators_x = data[label_indicators_x]
    indicators_y = data[label_indicators_y]
    
    if params is None:
        par1 = [np.random.uniform(-1,1) for i in range(len(label_indicators_x))]
        params0 = par1 + [np.random.uniform(0,0.5)]
    
        res = minimize(loglik, params0, args=(data, y, self_x, other_x, self_y, other_y,
              indicators_x, indicators_y),
                  method='BFGS', options={'disp': False, 'gtol':1e-7*len(data)})

    else:
        res = loglik(params, data, y, self_x, other_x, self_y, other_y,
              indicators_x, indicators_y)
    
    return res

In [179]:
[1,2] + [0]

[1, 2, 0]

In [180]:
dd = estim(data, 'choice_x', 'self_x', 'other_x',
     'self_y', 'other_y', ['s_x','r_x'], ['s_y','r_y'])

In [181]:
dd.x

array([0.0696009 , 0.2573429 , 0.01303487])

In [185]:
estim(test, 'choice_x', 'self_x', 'other_x',
     'self_y', 'other_y', ['s_x','r_x'], ['s_y','r_y'], params=dd.x)/len(test)

0.348184290452675

In [183]:
dd

      fun: 7133.97340220822
 hess_inv: array([[ 5.04685991e-05, -1.02089697e-05, -4.41139207e-08],
       [-1.02089697e-05,  4.77901478e-05, -1.48577669e-07],
       [-4.41139207e-08, -1.48577669e-07,  2.74067687e-08]])
      jac: array([-0.00030518,  0.0010376 , -0.00164795])
  message: 'Optimization terminated successfully.'
     nfev: 152
      nit: 24
     njev: 38
   status: 0
  success: True
        x: array([0.0696009 , 0.2573429 , 0.01303487])

In [187]:
data['self_x'].values

array([ 450,  450,  450, ..., 1060, 1060, 1060])